In [14]:
from datasets import load_dataset
# 加载数据集
ds = load_dataset("tiangler/cybersecurity_alarm_analysis")

# 看看有哪些 split
print(ds)

# 看前几条样本
ds["train"][:5]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 65134
    })
})


{'instruction': ['你是一名网络安全告警分析专家，请分析以下安全告警日志，重点分析payload字段，判断输入的告警日志是真实攻击还是误报。请按照以下格式给出回答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n<answer>标签中请直接回答"攻击"或"误报"。',
  '你是一名网络安全告警分析专家，请分析以下安全告警日志，重点分析payload字段，判断输入的告警日志是真实攻击还是误报。请按照以下格式给出回答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n<answer>标签中请直接回答"攻击"或"误报"。',
  '你是一名网络安全告警分析专家，请分析以下安全告警日志，重点分析payload字段，判断输入的告警日志是真实攻击还是误报。请按照以下格式给出回答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n<answer>标签中请直接回答"攻击"或"误报"。',
  '你是一名网络安全告警分析专家，请分析以下安全告警日志，重点分析payload字段，判断输入的告警日志是真实攻击还是误报。请按照以下格式给出回答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n<answer>标签中请直接回答"攻击"或"误报"。',
  '你是一名网络安全告警分析专家，请分析以下安全告警日志，重点分析payload字段，判断输入的告警日志是真实攻击还是误报。请按照以下格式给出回答：\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n<answer>标签中请直接回答"攻击"或"误报"。'],
 'input': ['{"write_date": "1732722399", "rule_version_str": "3.0.1122.14572", "dip_group": "省公司", "nid": "", "sip_group": "", "type": "信息泄露", "_origin": {"write_da

In [15]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("tiangler/cybersecurity_alarm_analysis")

# 提取 input 和 output
train_inputs = ds["train"]["input"]
train_labels = ds["train"]["output"]

# 看前几条
print(train_inputs[0])
print(train_labels[0])


{"write_date": "1732722399", "rule_version_str": "3.0.1122.14572", "dip_group": "省公司", "nid": "", "sip_group": "", "type": "信息泄露", "_origin": {"write_date": 1732722399, "vuln_harm": "服务器敏感信息一旦泄露会导致攻击者获得重要信息，增加攻击成功率。", "rule_version_str": "3.0.1122.14572", "hit_field": "", "dip": "172.31.192.209", "site_app": "", "hit_end": 0, "uri": "/gateway/support/oss/getPublicInputStream?filePath=/etc/anacrontab", "dport": 473, "vuln_name": "敏感信息扫描(机器学习)", "rsp_status": 200, "code_language": "", "solution": "请确认服务器是否存在相关文件，如果存在，可考虑迁移或删除敏感信息文件。", "hit_start": 0, "detail_info": "发现正在探测敏感信息文件，敏感的文件包括数据库文件、整站压缩文件、服务器或框架重要的配置文件、日志文件、压缩文件等。", "xff": "", "sip": "183.209.50.59", "vuln_desc": "发现正在探测敏感信息文件，敏感的文件包括数据库文件、整站压缩文件、服务器或框架重要的配置文件、日志文件、压缩文件等。", "public_date": "2020-11-25 19:44:12", "sport": 36228}, "detail_info": "发现正在探测敏感信息文件，敏感的文件包括数据库文件、整站压缩文件、服务器或框架重要的配置文件、日志文件、压缩文件等。", "file_md5": "", "rule_key": "webids", "connection": "keep-alive", "api": "zhny.hn.sgcc.com.cn/gateway/support/oss/getPublicInp

In [16]:
import json

# 提取关键信息当作文本特征
def extract_features(x):
    try:
        data = json.loads(x)
        # 挑一些代表性字段拼成字符串
        fields = [
            data.get("vuln_type",""),
            data.get("attack_type",""),
            data.get("uri",""),
            str(data.get("payload","")),
            data.get("detail_info","")
        ]
        return " ".join(fields)
    except Exception:
        return x  # 解析失败就用原始字符串

# 从整个数据集批量提取
texts = [extract_features(x) for x in ds["train"]["input"]]
labels = ds["train"]["output"]

print(texts[0][:500])  # 看前500个字符
print(labels[0])


敏感信息扫描(机器学习) 敏感信息/重要文件泄漏 /gateway/support/oss/getPublicInputStream?filePath=/etc/anacrontab {'req_header': 'R0VUIC9nYXRld2F5L3N1cHBvcnQvb3NzL2dldFB1YmxpY0lucHV0U3RyZWFtP2ZpbGVQYXRoPS9ldGMvYW5hY3JvbnRhYiBIVFRQLzEuMQ0KSG9zdDogemhueS5obi5zZ2NjLmNvbS5jbg0KQ29ubmVjdGlvbjoga2VlcC1hbGl2ZQ0Kc2VjLWNoLXVhOiAiQ2hyb21pdW0iO3Y9IjEyOCIsICJOb3Q7QT1CcmFuZCI7dj0iMjQiLCAiR29vZ2xlIENocm9tZSI7dj0iMTI4Ig0Kc2VjLWNoLXVhLW1vYmlsZTogPzANCnNlYy1jaC11YS1wbGF0Zm9ybTogIldpbmRvd3MiDQpVcGdyYWRlLUluc2VjdXJlLVJlcXVlc3RzOiAxDQpV
攻击


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 划分训练/测试
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 文本向量化
vec = TfidfVectorizer(max_features=8000)
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

# 训练分类器
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# 评估
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          攻击       0.97      0.58      0.73        64
          误报       1.00      1.00      1.00     12963

    accuracy                           1.00     13027
   macro avg       0.99      0.79      0.86     13027
weighted avg       1.00      1.00      1.00     13027



In [18]:
clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          攻击       0.63      1.00      0.78        64
          误报       1.00      1.00      1.00     12963

    accuracy                           1.00     13027
   macro avg       0.82      1.00      0.89     13027
weighted avg       1.00      1.00      1.00     13027



In [23]:
# 1) TF-IDF + 逻辑回归（已跑通）——加权
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, classification_report
import numpy as np, json

# texts, labels 见你上一步的提取；若还没做，沿用你已写的 extract_features()
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, stratify=labels, random_state=42)

vec = TfidfVectorizer(max_features=8000)
X_tr = vec.fit_transform(X_train); X_te = vec.transform(X_test)

clf = LogisticRegression(max_iter=2000, class_weight='balanced', n_jobs=-1)
clf.fit(X_tr, y_train)

# 2) 阈值扫一遍，选最合适的阈值（例如 F0.5 更偏向精准率，F2 更偏向召回）
pos = '攻击'
p = clf.predict_proba(X_te)[:, list(clf.classes_).index(pos)]
ths = np.linspace(0.8, 1, 1000)  # 你也可以自己定义一组阈值

def fbeta(p,y,th,beta=1.0):
    pred = np.where(p>=th, pos, '误报')
    pr, rc, f1, _ = precision_recall_fscore_support(y, pred, labels=[pos,'误报'],zero_division=0)
    # 取“攻击”这一类的P/R计算F_beta
    P, R = pr[0], rc[0]
    if P+R==0: return 0, P, R
    fbeta = (1+beta**2)*P*R/(beta**2*P+R+1e-12)
    return fbeta, P, R
# 计算每个阈值的指标
results = []
for t in ths:
    fbeta_val, P, R = fbeta(p, y_test, t, beta=1.0)
    results.append(((fbeta_val, P, R), t))

# 取最优
(best_f, bestP, bestR), best_th = max(results, key=lambda x: x[0][0])

print(f"Best threshold={best_th:.2f} | 攻击类 P={bestP:.3f}, R={bestR:.3f}, F1≈{best_f:.3f}")


Best threshold=0.93 | 攻击类 P=0.905, R=0.983, F1≈0.942


In [24]:
import numpy as np
from sklearn.metrics import precision_recall_curve

# 把标签二值化：攻击=1，其余=0
y_true = (np.array(y_test) == '攻击').astype(int)

# p 是你已经算好的“属于攻击”的概率
prec, rec, ths = precision_recall_curve(y_true, p)  # 注意: ths长度=prec长度-1

# 在所有阈值上计算 F1（或 Fβ）
beta = 1.0
F = (1+beta**2) * prec[:-1] * rec[:-1] / (beta**2 * prec[:-1] + rec[:-1] + 1e-12)

best_idx = np.nanargmax(F)
best_th = ths[best_idx]
bestP, bestR, bestF = prec[best_idx], rec[best_idx], F[best_idx]
print(f"最佳阈值={best_th:.6f} | P={bestP:.3f} R={bestR:.3f} F1={bestF:.3f}")


最佳阈值=0.939222 | P=0.905 R=0.983 F1=0.942


In [25]:
best_threshold = 0.939222  # 你算出来的值

# 对测试集打分并判别
y_pred_final = np.where(p >= best_threshold, '攻击', '误报')
print(classification_report(y_test, y_pred_final))


              precision    recall  f1-score   support

          攻击       0.90      0.97      0.93        58
          误报       1.00      1.00      1.00     12969

    accuracy                           1.00     13027
   macro avg       0.95      0.98      0.97     13027
weighted avg       1.00      1.00      1.00     13027



In [5]:
import joblib
with open('tfidf_vec.pkl', 'rb') as f:
    vec = joblib.load(f)

with open('logreg_model.pkl', 'rb') as f:
    model = joblib.load(f)


print(vec)  # 看看向量化器
print(model)  # 看看模型
# 假设 vec, model 已经用了 joblib.load 加载并可用
import numpy as np
import csv

# 获取 feature names（兼容老/新 sklearn）
try:
    feature_names = vec.get_feature_names_out()
except:
    feature_names = None

if feature_names is None:
    inv = {i: t for t, i in vec.vocabulary_.items()}
    feature_names = [inv.get(i, "") for i in range(len(inv))]

coef = getattr(model, "coef_", None)
classes = getattr(model, "classes_", None)
if coef is None or classes is None:
    print("模型没有 coef_ 或 classes_ 属性；无法导出系数。")
else:
    print("coef_ shape:", coef.shape, "classes:", classes)

    # 处理二分类单行系数情况
    rows = []  # 每项: (class_name, feature_index, feature_name, weight)
    if coef.shape[0] == len(classes):
        # 每行对应一个类（多类或 sklearn 已返回每类系数）
        for c_idx, cls in enumerate(classes):
            for i, fname in enumerate(feature_names):
                rows.append([cls, i, fname, float(coef[c_idx, i])])
    elif coef.shape[0] == 1 and len(classes) == 2:
        # sklearn 二分类常见：coef_[0] 对应 classes_[1]（正类）
        pos_cls = classes[1]
        neg_cls = classes[0]
        w = coef[0]
        # 写入正类（classes_[1]）的权重（直接写 coef）
        for i, fname in enumerate(feature_names):
            rows.append([pos_cls, i, fname, float(w[i])])
        # 如果想要为负类也写一列，可以把权重取反写入（可选）
        # for i, fname in enumerate(feature_names):
        #     rows.append([neg_cls, i, fname, float(-w[i])])
        print(f"注意：模型为二分类，coef_ shape=(1, n_features)。该单行系数对应类 '{pos_cls}'。如果需要，'{neg_cls}' 的系数等于取反。")
    else:
        # 其他不常见情况，尽量写出现有行并标注
        for r_idx in range(coef.shape[0]):
            cls_name = classes[r_idx] if r_idx < len(classes) else f"row{r_idx}"
            for i, fname in enumerate(feature_names):
                rows.append([cls_name, i, fname, float(coef[r_idx, i])])
        print("注意：coef 行数 与 classes 数不匹配，已按现有行写出。")

    # 导出到 CSV（按行写）
    out_csv = "logreg_coefs_fixed.csv"
    with open(out_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["class", "feature_index", "feature_name", "weight"])
        writer.writerows(rows)
    print("已导出系数到", out_csv)

    # 同时打印 top-k 正负特征（对正类）
    target_class = '攻击'
    if target_class in classes:
        # 得到正类对应的权重向量（兼容二分类 single-row 情况）
        if coef.shape[0] == len(classes):
            cls_idx = list(classes).index(target_class)
            weights = coef[cls_idx]
        elif coef.shape[0] == 1 and len(classes) == 2:
            # 单行 coef 对应 classes_[1]
            if classes[1] == target_class:
                weights = coef[0]
            else:
                # target_class 是负类，则权重为 -coef[0]
                weights = -coef[0]
        else:
            # 兜底
            cls_idx = list(classes).index(target_class) if target_class in classes and list(classes).index(target_class) < coef.shape[0] else 0
            weights = coef[cls_idx]

        weights = np.asarray(weights).ravel()
        top_pos_idx = np.argsort(weights)[-20:][::-1]
        top_neg_idx = np.argsort(weights)[:20]

        print("Top positive features for class '{}':".format(target_class))
        for i in top_pos_idx:
            print(f"  {feature_names[i]}\t{weights[i]:.6f}")
        print("Top negative features for class '{}':".format(target_class))
        for i in top_neg_idx:
            print(f"  {feature_names[i]}\t{weights[i]:.6f}")
    else:
        print(f"目标类 '{target_class}' 不在 classes 中：", classes)

TfidfVectorizer(max_features=8000)
LogisticRegression(class_weight='balanced', max_iter=2000)
coef_ shape: (1, 8000) classes: ['攻击' '误报']
注意：模型为二分类，coef_ shape=(1, n_features)。该单行系数对应类 '误报'。如果需要，'攻击' 的系数等于取反。
已导出系数到 logreg_coefs_fixed.csv
Top positive features for class '攻击':
  www	4.014380
  218	3.321932
  jsp	3.201631
  2e	3.173770
  30001	3.021761
  新加坡	2.910784
  cve	2.907815
  发现nmap探测行为	2.609954
  68	2.520490
  agent	2.518982
  146	2.456652
  机器学习	2.375667
  windows	2.278071
  76	2.260736
  realtek	2.091920
  hn	1.975539
  160	1.900512
  git	1.874087
  compatible	1.862811
  js	1.832468
Top negative features for class '攻击':
  172	-4.194921
  null	-3.766972
  发现使用远程连接工具向日葵	-3.028038
  isc	-2.933844
  22	-2.541026
  31	-2.539155
  200	-2.425912
  617934985	-2.409219
  dns	-2.384798
  client_group_ip	-2.275754
  19	-2.106664
  qbjk	-2.051079
  3b	-2.008617
  192	-2.004428
  group	-1.959723
  224	-1.861707
  nx	-1.817841
  132	-1.807025
  弱口令	-1.794730
  发现有弱口令登录行为	-1.790036
